In [37]:
#!pip install statsmodels

# Display plots directly in the notebook
%matplotlib inline
#!conda install -c conda-forge geopandas

# Import libraries
import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib as mpl

import seaborn as sns

import patsy
#import statsmodels.api as sm

In [71]:
yelp_SD = pd.read_csv('../data/yelp_SD_parks.csv')
park_df = gpd.read_file('../data/parks_datasd.geojson')
#access_df = pd.read_excel('../data/hci_accesstoparks_469_ca_re_co_cd_pl_ct-7-3-2017-ada.xlsx')

In [72]:
#access_df.columns

In [73]:
#access_df = access_df.loc[access_df['region_code']==9.0,
                         # ['ind_id','race_eth_name','numerator','estimate']]

In [74]:
#access_df.iloc[:10]

In [75]:
#drop columns don't need
park_df = park_df.drop(columns=['objectid','name','location','park_type','owner','geometry'])
park_df = park_df.rename(columns={'alias':'name', 'gis_acres':'acres'})

In [76]:
park_df.head()

,name,acres
0,South Carlsbad State Beach,115.895878
1,Torrey Pines State Beach,67.294309
2,Ruocco Park,3.312526
3,Tuna Harbor Park,0.639035
4,San Diego Bayfront Park,3.669272


In [77]:
yelp_SD = yelp_SD.drop(columns=['phone','id','url','longitude','latitude','is_closed'])

In [78]:
# Transform address to zipcode
def address_to_zip(string):
    zip = []
    for str in string:
        if str[-5] == '9':
            zip.append(str[-5:]) 
        else:
            zip.append(str)
          
    return zip

In [79]:
yelp_SD['zipcode'] = address_to_zip(yelp_SD['address'])

In [80]:
yelp_SD = yelp_SD.loc[:,['name','zipcode','rating','review_count']]
yelp_SD['review_count'].describe()

count      833.000000
mean       201.246098
std        744.775857
min          1.000000
25%          6.000000
50%         22.000000
75%        109.000000
max      14195.000000
Name: review_count, dtype: float64

In [88]:
yelp_SD = yelp_SD.sort_values(by='rating', ascending=False)
df_drop = yelp_SD[yelp_SD['review_count']<= 6]
yelp_SD = yelp_SD.drop(df_drop.index)
yelp_SD.head()

,name,zipcode,rating,review_count
0,Balboa Park,92101,5.0,2105
207,North Coast Repertory Theatre,92075,5.0,34
186,San Diego Burial At Sea,92106,5.0,26
283,The Color Collective Salon,92103,5.0,40
240,Self Realization Fellowship Hermitage & Medita...,92024,5.0,345


In [85]:
print(park_df.shape, yelp_SD.shape)

(2769, 2) (532, 4)


In [114]:
park_df.loc[park_df['name']== 'Zeglio Custom Clothiers']

,name,acres


In [107]:
yelp_SD['name'].iloc[20:30]

669                      Quality Mobile Screens
279                                   CurlCraft
717                     Zeglio Custom Clothiers
530                       San Diego Rowing Club
526                        St Therese of Carmel
633             Superb Mortgage and Real Estate
613                             Ceramic Heights
540                    Aall In Limo & Party Bus
57     Mount Soledad National Veterans Memorial
724                 Mesa Clothing & Alterations
Name: name, dtype: object

In [118]:
df = pd.merge(yelp_SD, park_df, on='name')
df

,name,zipcode,rating,review_count,acres
0,Balboa Park,92101,5.0,2105,1089.476460
1,Kellogg Park,92037,5.0,17,8.024891
2,Sunset Cliffs Natural Park,92107,5.0,935,67.109374
3,4S Ranch Community Park,92127,5.0,17,3.241830
4,Otay Lakes County Park,91915,4.5,21,10.349566
...,...,...,...,...,...
114,Poway Dog Park,92064,3.5,82,1.838939
115,Pantoja Park,92101,3.5,26,2.148412
116,Clay Park,92115,3.5,16,2.407481
117,El Toyon Park,91950,3.0,17,22.868727


In [120]:
df2 = pd.concat([park_df, yelp_SD],axis=1,join='inner')
df2

,name,acres,name,zipcode,rating,review_count
0,South Carlsbad State Beach,115.895878,Balboa Park,92101,5.0,2105
1,Torrey Pines State Beach,67.294309,Civita Park,92108,4.5,46
2,Ruocco Park,3.312526,Waterfront Park,92101,4.5,242
3,Tuna Harbor Park,0.639035,Trolley Barn Park,92116,4.5,102
4,San Diego Bayfront Park,3.669272,Bay View Park,92118,5.0,42
...,...,...,...,...,...,...
825,Crest Canyon Open Space Park,4.061640,Qualcomm Stadium Trolley Station,92108,3.0,45
828,Chollas Radio System Open Space,5.719391,Del Cerro - San Diego,92120,4.5,12
829,Canyon Hills Open Space,0.112745,San Diego Metropolitan Transit System,92101,2.5,268
830,San Pasqual Trails Open Space,161.542345,Petco Park,92101,4.0,1667
